In [5]:
#Lets import necessory libraries
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import datetime
import keras

# Create model function

In [6]:
def create_model():
    return tf.keras.models.Sequential([  # Define a sequential model
        tf.keras.layers.Flatten(input_shape=(28, 28)),  # Flatten 28x28 images to a 1D array
        tf.keras.layers.Dense(512, activation='relu'),  # Add a dense layer with 512 neurons and ReLU activation
        tf.keras.layers.Dropout(0.2),  # Add dropout layer to prevent overfitting
        tf.keras.layers.Dense(10, activation='softmax')  # Output layer with 10 neurons (one for each class), using softmax activation
    ])

# Setup summary writers

In [7]:

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # Get current time for logging
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'  # Directory for training logs
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'  # Directory for testing logs
train_summary_writer = tf.summary.create_file_writer(train_log_dir)  # Create writer for training logs
test_summary_writer = tf.summary.create_file_writer(test_log_dir)  # Create writer for testing logs

In [8]:
# Prepare datasets

In [9]:

mnist = tf.keras.datasets.mnist  # Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()  # Split into train and test data
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize the pixel values to [0, 1]
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(60000).batch(64)  # Create a batched and shuffled dataset for training
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)  # Create a batched dataset for testing

# Define loss and metrics

In [10]:

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()  # Define the loss function (sparse categorical crossentropy)
train_loss = tf.keras.metrics.Mean()  # Track the mean loss for training
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()  # Track accuracy for training
test_loss = tf.keras.metrics.Mean()  # Track the mean loss for testing
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()  # Track accuracy for testing


In [11]:
# Define training and testing steps
optimizer = tf.keras.optimizers.Adam()  # Define the optimizer (Adam)

## Using Custom Training Loops with tf.summary

### For more control, you might use tf.GradientTape and tf.summary to log custom metrics.

    1. Convert Data to Dataset
        Convert data to tf.data.Dataset for efficient batching.

## What is tf.GradientTape?
### tf.GradientTape is a tool in TensorFlow that helps you calculate how much the output of your model changes when you adjust its inputs or weights. It’s like a way to see how the model is learning during training.

## What is tf.summary?
### tf.summary is used to save information about how your model is training. You can think of it as a way to take notes during training so you can look back later and understand how well your model was learning.

## Custom Training Loops
### When you train a model, TensorFlow usually does everything for you automatically. But sometimes, you want more control to do things your way. For example, you might want to try different ways of updating your model or add custom calculations.

## Putting It Together
### When you use tf.GradientTape and tf.summary together, you can:

#### Manually control how the model learns: You can decide how to update the model step by step, instead of letting TensorFlow do it automatically.

#### Log custom metrics: You can save detailed information about how well the model is learning, such as how accurate it is or how much it improves with each step.

            

In [12]:
@tf.function  # Decorator to optimize the training function
def train_step(model, x_train, y_train):
    with tf.GradientTape() as tape:  # Record operations for automatic differentiation
        predictions = model(x_train, training=True)  # Make predictions using the model
        loss = loss_object(y_train, predictions)  # Calculate the loss
    grads = tape.gradient(loss, model.trainable_variables)  # Compute the gradients of the loss w.r.t. model variables
    optimizer.apply_gradients(zip(grads, model.trainable_variables))  # Update model variables

    train_loss(loss)  # Update training loss metric
    train_accuracy(y_train, predictions)  # Update training accuracy metric

@tf.function  # Decorator to optimize the testing function
def test_step(model, x_test, y_test):
    predictions = model(x_test)  # Make predictions on the test set
    loss = loss_object(y_test, predictions)  # Calculate the loss

    test_loss(loss)  # Update testing loss metric
    test_accuracy(y_test, predictions)  # Update testing accuracy metric

In [13]:
# Train and evaluate the model
model = create_model()  # Instantiate the model
EPOCHS = 5  # Define the number of epochs

for epoch in range(EPOCHS):
    # Training loop
    for (x_train_batch, y_train_batch) in train_dataset:  # Iterate over the training dataset
        train_step(model, x_train_batch, y_train_batch)  # Perform a training step
    
    # Log training metrics
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)  # Log the training loss
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)  # Log the training accuracy

    # Testing loop
    for (x_test_batch, y_test_batch) in test_dataset:  # Iterate over the testing dataset
        test_step(model, x_test_batch, y_test_batch)  # Perform a testing step
    
    # Log testing metrics
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)  # Log the testing loss
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)  # Log the testing accuracy

    # Print metrics for this epoch
    print(f'Epoch {epoch+1}, Loss: {train_loss.result()}, Accuracy: {train_accuracy.result()*100}, Test Loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}')

    # Reset metrics for the next epoch
    train_loss.reset_states()  # Reset training loss
    test_loss.reset_states()  # Reset testing loss
    train_accuracy.reset_states()  # Reset training accuracy
    test_accuracy.reset_states()  # Reset testing accuracy

Epoch 1, Loss: 0.24631930887699127, Accuracy: 92.92333221435547, Test Loss: 0.12051034718751907, Test Accuracy: 96.4800033569336
Epoch 2, Loss: 0.105633944272995, Accuracy: 96.83999633789062, Test Loss: 0.08637232333421707, Test Accuracy: 97.3699951171875
Epoch 3, Loss: 0.07129782438278198, Accuracy: 97.8133316040039, Test Loss: 0.07112293690443039, Test Accuracy: 97.7199935913086
Epoch 4, Loss: 0.05508251488208771, Accuracy: 98.25666809082031, Test Loss: 0.06502792984247208, Test Accuracy: 97.82999420166016
Epoch 5, Loss: 0.04291757196187973, Accuracy: 98.67832946777344, Test Loss: 0.06042737513780594, Test Accuracy: 98.07999420166016


# Check your log directory

In [ ]:
!tensorboard --logdir logs/gradient_tape

2024-08-16 12:54:36.433827: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-16 12:54:36.551957: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-16 12:54:36.552036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-16 12:54:36.554619: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-16 12:54:36.572671: I tensorflow/core/platform/cpu_feature_guar

# Delete your Directory

In [ ]:
import tensorflow as tf
import datetime
import shutil
import os

# Clear previous logs
log_dir = 'logs/gradient_tape/'
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)

# Continue with the rest of your code...
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = log_dir + current_time + '/train'
test_log_dir = log_dir + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


# Manually Delete the Log Directory


In [24]:
!rm -rf logs/gradient_tape/


In [25]:
!rmdir /S /Q logs\gradient_tape


rmdir: failed to remove '/S': No such file or directory
rmdir: failed to remove '/Q': No such file or directory
rmdir: failed to remove 'logsgradient_tape': No such file or directory


# After Deleting Logs


In [ ]:
!tensorboard --logdir=logs/gradient_tape/
